In [ ]:
import numpy as np
from numpy.linalg import norm, det, inv, LinAlgError, lstsq
from scipy.spatial import Delaunay, Voronoi, voronoi_plot_2d
from scipy.interpolate import LinearNDInterpolator
from matplotlib import pyplot as plt

from numba import jit, njit

from tqdm import trange

from greenGauss import gaussDiv, gaussGrad

In [ ]:
x_n = np.random.rand(5, 2)

u_n = np.random.rand(5)

pt = np.array([0.5, 0.5])

dx = x_n - pt

R = np.sqrt(np.sum(dx**2, axis=1))

A = np.concatenate([R[:, np.newaxis], (R*dx.T).T], axis=1)
b = (R*u_n.T).T

x, _, _, _ = lstsq(A,b)


In [ ]:
@njit
def _lstsq_grad(points, vertices, q, pts, ridge, neighbours, pt):
    """calculate least squares gradient based on nearest neighbours
       e.g. GG-LSQ https://doi.org/10.1186/s42774-019-0020-9"""
    cells = np.array(list(set(pts) | set(neighbours)))

    x_n = points[cells]

    q_n = q[cells]

    dx = x_n - pt

    Rinv = 1/np.sqrt(np.sum(dx**2, axis=1))
    
    A = np.zeros((dx.shape[0], dx.shape[1]+1))
    A[:, 0] = Rinv
    A[:, 1:] = (Rinv*dx.T).T
    b = (Rinv*q_n.T).T

    x, _, _, _ = lstsq(A, b)

    return x[1:]


def face_grad(mesh, u, dim='2d'):
    
    n_regions = len(mesh.regions)
    n_ridges = len(mesh.ridge_points)
    n_points = len(mesh.points)

    # collect neighbours of each point as a list of lists
    neighbours = [[] for i in range(n_points)]
    n_neighbours = np.zeros(n_points)
    for points in mesh.ridge_points:
        neighbours[points[0]].append(points[1])
        neighbours[points[1]].append(points[0])
        n_neighbours[points[0]] += 1
        n_neighbours[points[1]] += 1

    # iterate over faces to calculate flux in or out
    out = np.zeros((n_ridges, mesh.points.shape[1]))

    for ridge in trange(n_ridges):
        vertices = mesh.ridge_vertices[ridge]
        
        # exclude infinite vertices
        if -1 in vertices:
            continue

        point_idx = mesh.ridge_points[ridge]
        
        face_centr = mean(mesh.vertices[vertices], axis=0)  # centroid of current face

        shared_neighbours = list(
            set(neighbours[point_idx[0]])
            & set(neighbours[point_idx[1]])
        )
        if len(shared_neighbours) == 0:
            shared_neighbours = list(
                (set(neighbours[point_idx[0]]) - {point_idx[1]})
                | (set(neighbours[point_idx[1]]) - {point_idx[0]})
            )
        
        out[ridge] = _lstsq_grad(
            mesh.points,
            mesh.vertices,
            u,
            point_idx,
            ridge,
            shared_neighbours,
            face_centr
        )

    return out


def _establish_neighbours(neighbours, pts):
    shared_neighbours = list(
        (set(neighbours[pts[0]])
        & set(neighbours[pts[1]]))
    )
    if len(shared_neighbours) == 0:
        shared_neighbours = list(
            (set(neighbours[pts[0]]) - {pts[1]})
            | (set(neighbours[pts[1]]) - {pts[0]})
        )
    return shared_neighbours


@njit
def mean(x, axis=0):
    return np.sum(x, axis) / x.shape[axis]



## 2D gradient test

Evaluate function with known gradient

In [ ]:
x_raw = np.linspace(-1, 1, 21)
y_raw = np.linspace(-1, 1, 21)

xx, yy = np.meshgrid(x_raw, y_raw)

xy = np.transpose([xx.flatten(), yy.flatten()])
R = np.array([
    [np.cos(np.pi/4), np.sin(np.pi/4)],
    [-np.sin(np.pi/4), np.cos(np.pi/4)],
])

xy = xy@R

mesh = Voronoi(xy)

In [ ]:
plt.figure()
voronoi_plot_2d(mesh,)
plt.gca().set_aspect('equal')


In [ ]:
n_ridges = len(mesh.ridge_vertices)
face_xy = np.zeros((n_ridges, 2))

for i in range(n_ridges):
    vertices = [vertex for vertex in mesh.ridge_vertices[i] if vertex!=-1]
    face_xy[i] = np.mean(mesh.vertices[vertices], axis=0)

x, y = face_xy.T

p = xy[:,0]*xy[:,1]
grad_p = np.stack([y, x], axis=1)


In [ ]:
dp = face_grad(mesh,p)

In [ ]:
err = dp - grad_p

fig, ax = plt.subplots(2,3, sharex=True, sharey=True)

tcf = ax[0,0].tricontourf(x, y, grad_p[:, 0])
fig.colorbar(tcf, ax=ax[0,0], shrink=0.5)

tcf = ax[1,0].tricontourf(x, y, grad_p[:, 1])
fig.colorbar(tcf, ax=ax[1,0], shrink=0.5)

tcf = ax[0,1].tricontourf(x, y, dp[:, 0])
fig.colorbar(tcf, ax=ax[0,1], shrink=0.5)

tcf = ax[1,1].tricontourf(x, y, dp[:, 1])
fig.colorbar(tcf, ax=ax[1,1], shrink=0.5)

tcf = ax[0,2].tricontourf(x, y, err[:, 0])
fig.colorbar(tcf, ax=ax[0,2], shrink=0.5)

tcf = ax[1,2].tricontourf(x, y, err[:, 1])
fig.colorbar(tcf, ax=ax[1,2], shrink=0.5)

for axes in ax:
    for axax in axes:
        
        axax.set_aspect('equal')
fig.tight_layout()

In [ ]:
err_points = norm(err, axis=1) > 1e-2

In [ ]:
_ = plt.hist(err.flatten(), bins=100)

In [ ]:
from matplotlib.colors import LogNorm

plt.tricontourf(x[inner_pts],  y[inner_pts], np.abs((pt_du[inner_pts] - divU[inner_pts])),
                #norm=LogNorm(),
                levels=16)
plt.colorbar()
xlim = plt.xlim()
ylim = plt.ylim()
plt.plot(mesh.vertices[:, 0], mesh.vertices[:, 1], ',', color='orange')

plt.xlim(xlim)
plt.ylim(ylim)

## 3D divergence test

In [ ]:
x_raw = np.linspace(-1, 1, 41)
y_raw = np.linspace(-1, 1, 41)
z_raw = np.linspace(-1, 1, 41)

xx, yy, zz = np.meshgrid(x_raw, y_raw, z_raw)

xyz = np.transpose([xx.flatten(), yy.flatten(), zz.flatten()])
R = np.array([
    [np.cos(np.pi/4), np.sin(np.pi/4), 0],
    [-np.sin(np.pi/4), np.cos(np.pi/4), 0],
    [0, 0, 1]
])

xyz = xyz@R

mesh = Voronoi(xyz)

In [ ]:
n_ridges = len(mesh.ridge_vertices)
face_xyz = np.zeros((n_ridges, 3))

for i in range(n_ridges):
    vertices = [vertex for vertex in mesh.ridge_vertices[i] if vertex!=-1]
    face_xyz[i] = np.mean(mesh.vertices[vertices], axis=0)

x, y, z = face_xyz.T

p = xyz[:,0]*xyz[:,1]*xyz[:,2]
grad_p = np.stack([y*z, x*z, x*y], axis=1)

dp = face_grad(mesh,p)

In [ ]:
inner_regions = np.array([i for i, region in enumerate(mesh.regions) if -1 not in region])

inner_pts = [-1 not in mesh.regions[region] for region in mesh.point_region]

In [ ]:
err = dp - grad_p

err_points = norm(err, axis=1) > 1e-2

In [ ]:
_ = plt.hist(grad_p, bins=100)

In [ ]:
n_faces = 0
n_outer = 0
for i in range(n_ridges):
    n_faces += 1
    if -1 in mesh.ridge_vertices[i]:
        n_outer += 1

_ = plt.hist(np.log10(np.abs(err)+1e-6), bins=100, cumulative=True, density=True)
plt.axhline(1 - n_outer / n_faces)